In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

In [2]:
df=pd.read_csv(r'D:/PythonProject/fakeNews/train.csv')

In [3]:
df.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [4]:
df.groupby("Labels").count()

,Text,Text_Tag
Labels,,
0,1654,1654
1,1995,1993
2,2114,2114
3,1962,1962
4,839,839
5,1676,1676


In [5]:
df.describe()

,Labels
count,10240.000000
mean,2.328613
std,1.650933
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,5.000000


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.Text, df.Labels, random_state=4,test_size=0.33)

# TF-IDF

In [7]:
vectoriser = TfidfVectorizer(use_idf=True, ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [8]:
X_tfidf = vectoriser.transform(X_train)
X_test_tfidf = vectoriser.transform(X_test)

In [9]:
clf_tf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tf.fit(X_tfidf, y_train)

y_predicted_tf = clf_tf.predict(X_test_tfidf)
train_mean = np.mean(y_predicted_tf, axis=0)
print(train_mean)
train_mean2 = np.std(y_predicted_tf, axis=0)
print(train_mean2)

2.2517751479289942
1.6216672103602277


# TF

In [10]:
vectoriser = TfidfVectorizer(use_idf=False, ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2), use_idf=False)

In [11]:
X_tfidf = vectoriser.transform(X_train)
X_test_tfidf = vectoriser.transform(X_test)

In [12]:
clf_tf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tf.fit(X_tfidf, y_train)

y_predicted_tf = clf_tf.predict(X_test_tfidf)
train_mean = np.mean(y_predicted_tf, axis=0)
print(train_mean)
train_mean2 = np.std(y_predicted_tf, axis=0)
print(train_mean2)

2.2707100591715976
1.6360672552995217


# LSI

In [13]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [14]:
vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True,max_features=1000)

In [15]:
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10)
svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', lsa_model)])

lsa_top = svd_transformer.fit_transform(X_train)

lsa_test = svd_transformer.transform(X_test)

In [16]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(lsa_top, y_train)

y_predicted_tfidf = clf_tfidf.predict(lsa_test)
train_mean = np.mean(y_predicted_tfidf, axis=0)
print(train_mean)
train_mean2 = np.std(y_predicted_tfidf, axis=0)
print(train_mean2)

3.071597633136095
1.711142855989489
